In [25]:
import sagemaker
import boto3
import pandas
import numpy
import io

In [17]:
# If you are running the script locally uncomment this line
# role = "arn-sagemaker-role-for-the-user"

# If you are running this notebook in Sagemaker Notebook, uncomment this line
# role = sagemaker.get_execution_role()

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
runtime = boto3.client("runtime.sagemaker")

endpoint_name_1 = "lclm-tricore-modernbert-histogram-endpoint"
# Get the model_package_arn
modelpackage_arn = "arn:aws:sagemaker:us-east-1:258916827218:model-package/lclm-tricore-histogram"

# create a deployable model for damage inspection model package.
model = sagemaker.ModelPackage(
    role=role,
    model_package_arn=modelpackage_arn,
    sagemaker_session=sagemaker_session,
)

In [ ]:
# Deploy the model
predictor = model.deploy(1, "ml.g5.xlarge", endpoint_name=endpoint_name_1)

In [ ]:
data = pandas.read_csv('../data/tricore_dataset.csv')
print(data.head())
data_bytes = data.to_csv(None, index=False)

In [ ]:
# Perform a prediction
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name_1,
    ContentType="text/csv",
    Body=data_bytes
)
array = numpy.load(io.BytesIO(response['Body'].read()))
# View the prediction
print(array)

In [ ]:
# Delete the endpoint and the model
model.sagemaker_session.delete_endpoint(endpoint_name_1)
model.sagemaker_session.delete_endpoint_config(endpoint_name_1)
model.delete_model()